In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

: 

# Exploratory Analysis with Linear Regressions #
Objective: Examine the relationship between median SAT score and the rate of Undergraduate Degree completion at universities across the United States 

### Load .csv into DataFrame ###

In [ ]:
# Read csv
df = pd.read_csv('cc_institution_details.csv',encoding = 'unicode_escape')

# Drop rows with NaN values in these two columns
# Reference - https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html
df.dropna(subset=['med_sat_value','grad_100_value'],inplace=True)

df[['chronname','med_sat_value','grad_100_value']].head(5)


DataFrame description and info 

In [ ]:
df.describe()

In [ ]:
df.info()

### Identify independent variable ###

In [ ]:
X = df['med_sat_value'].values.reshape(-1,1)
X

### Identify dependent variable ###

In [ ]:
Y = df['grad_100_value'].values.reshape(-1,1)
Y

### Split Data into Training/Testing & Plot points ###

In [ ]:
# Split the data into training and testing portions
x_train, x_test, y_train, y_test = train_test_split(X,Y, train_size=.8, test_size=.2, random_state=100)

# Plot the best fit line and scatter-plot of data
m, b = np.polyfit(df['med_sat_value'], df['grad_100_value'], 1)
plt.plot(X, m*X + b)
plt.scatter(x_train, y_train, color='red')

# Assign labels 
plt.xlabel('Median SAT Score')
plt.ylabel=('Graduation Rate')
plt.title('Training Data')

plt.show()

### Fit Linear Model and Make Predictions ###

In [ ]:
lm = LinearRegression()
lm.fit(x_train, y_train)
y_predict = lm.predict(x_test)


training_acc = round(lm.score(x_train, y_train)*100, 2)
testing_acc = round(lm.score(x_test, y_test)* 100, 2)
print(f'Training Accuracy {training_acc}%')
print(f'Test Accuracy {testing_acc}%')

### Conclusions ###
* Since the accuracy of the model prediction is < 60%, we can make the assumption that there is not a strong relationship between Median SAT Score and Graduation Rate
* From the scatter plot we can see that the residuals trend closer to the best-fit line as the median SAT score increases
* There are several outliers in the data, with relatively high graduation rates at low median SAT scores and vice versa